# Volatility Managed Portfolios

# <u> Summary : 

- [1) Parametric Portfolio](#1)
    - [1.1) Factors](#1.1)
    - [1.2) Optimal mean Variance $\theta$ for parametric portfolio](#1.2)
    - [1.3) Sharpe ratios of the factors and of the parametric portfolio](#1.3)
    - [1.4) Getting the 'real' weights](#1.4)
- [2) Volatility Managed Portfolio](#2)
    - [2.1) Volatility timed factors](#2.1)
    - [2.2) Mean Variance Optimization](#2.2)
- [3) Comparison and Conclusion](#3)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt

<a id='1'></a>
## <u> 1.Parametric Portfolio

In [2]:
data_excel=pd.read_excel("4.1- Data.xlsx")

<a id='1.1'></a>
### <u> 1.1 Factors


    Market (MKT): The market factor captures the overall return of the stock market relative to the risk-free rate. It reflects the systematic risk that cannot be diversified, and its premium compensates investors for taking on market risk.

    Small Minus Big (SMB): This factor represents the size effect, measuring the difference in returns between small-cap and large-cap stocks. Small-cap stocks tend to outperform because they are riskier and less liquid, attracting a premium.

    High Minus Low (HML): The value factor captures the difference in returns between value stocks (high book-to-market ratio) and growth stocks (low book-to-market ratio). Value stocks often outperform because they are perceived as undervalued and carry a higher risk of financial distress.

    Robust Minus Weak (RMW): This profitability factor measures the difference in returns between firms with robust profitability and those with weak profitability. More profitable companies tend to generate higher returns as they are financially healthier and more stable.

    Conservative Minus Aggressive (CMA): This factor captures the investment effect, comparing companies with conservative investment policies to those with aggressive ones. Firms with conservative investment policies often achieve higher returns due to better capital allocation and risk management.

    Momentum (UMD): The momentum factor identifies the tendency of stocks that have performed well in the past to continue performing well in the short term. This persistence is often attributed to behavioral biases and slow information diffusion.

    Return on Equity (ROE): This factor measures a firm's profitability relative to shareholder equity, with higher ROE firms typically offering higher returns. It serves as an indicator of operational efficiency and financial performance.

    Investment (IA): This factor reflects the tendency of firms with lower asset growth (investment) to outperform those with higher growth. Overinvestment may lead to inefficient capital allocation and lower future returns.

    Betting Against Beta (BAB): The BAB factor captures the return difference between low-beta and high-beta stocks. Low-beta stocks tend to outperform because investors overpay for high-beta stocks due to leverage constraints and behavioral biases.

<a id='1.2'></a>
### <u> 1.2 Optimal mean Variance $\theta$ for parametric portfolio

We assume that we have a risk aversion of $\gamma = 5 $

In [3]:
gamma = 5

note that this is in sample on the whole data

In [4]:
data=data_excel.set_index('Dates')
data.head()

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
196702,0.0078,0.0334,-0.0217,0.0194,-0.0094,0.0356,0.035317,-0.002064,0.0262
196703,0.0399,0.0163,0.0031,0.0090,-0.0151,0.0142,0.018876,-0.016933,0.0081
196704,0.0389,0.0062,-0.0264,0.0243,-0.0375,0.0064,0.010983,-0.029519,0.0171
196705,-0.0433,0.0198,0.0080,-0.0175,0.0161,0.0067,0.005234,0.024686,0.0201
196706,0.0241,0.0596,0.0096,-0.0064,-0.0239,0.0603,0.002945,-0.021700,-0.0163


In [5]:
# Step 1: Calculate the mean vector (mu) of factor returns
mu = data.mean()

# Step 2: Calculate the covariance matrix (Sigma) of factor returns
Sigma = data.cov()

# Step 3: Compute the optimal theta vector using the formula
theta = np.linalg.inv(Sigma) @ mu / gamma

# Convert theta to a pandas Series for better readability
theta_series = pd.Series(theta, index=data.columns, name="Optimal Weights")

# Display the results
print("Optimal Weights (Theta Vector):")
print(theta_series)


Optimal Weights (Theta Vector):
Market    1.106184
SMB       0.955817
HML      -0.155005
RMW       0.207042
CMA       0.153447
UMD       0.216184
ROE       1.773400
IA        2.726123
BAB       0.778341
Name: Optimal Weights, dtype: float64


In [6]:
def parametric_MV(excess_returns, gamma=1, long_only=False):
    
    mu = excess_returns.mean()
    Sigma = excess_returns.cov()
    
    if long_only==False:
        theta= np.linalg.inv(Sigma) @ mu / gamma
        theta_scaled=theta/((np.linalg.inv(Sigma) @ mu / gamma).dot((np.ones(len(mu)).T)))
        return pd.Series(theta_scaled, index=excess_returns.columns, name="Optimal Weights")
    
    else:
        
        def objective(weights):
            portfolio_return = weights.T @ mu
            portfolio_risk = np.sqrt(weights.T @ Sigma @ weights)
            return gamma/2*portfolio_risk -portfolio_return 
        
        constraints = ({"type": "eq", "fun": lambda weights: np.sum(weights) - 1})
        bounds = tuple((0,999) for _ in range(len(Sigma)))
        initial_weights = np.ones(len(mu)) / len(mu)
    
        theta = minimize(objective, initial_weights, method="SLSQP", constraints=constraints,bounds = bounds,).x
        
        return pd.Series(theta, index=excess_returns.columns, name="Optimal Weights")

In [7]:
# Theta without constraint
theta=parametric_MV(data,5)
theta

Market    0.142521
SMB       0.123148
HML      -0.019971
RMW       0.026675
CMA       0.019770
UMD       0.027853
ROE       0.228486
IA        0.351235
BAB       0.100282
Name: Optimal Weights, dtype: float64

In [8]:
# Theta with long only constraint
theta_LO=parametric_MV(data,5,True)
theta_LO

Market    0.106065
SMB       0.177738
HML       0.007074
RMW       0.177711
CMA       0.181635
UMD       0.021253
ROE       0.147770
IA        0.180755
BAB       0.000000
Name: Optimal Weights, dtype: float64

<a id='1.3'></a>
### <u> 1.3 Sharpe ratios of the factors and of the parametric portfolio

In [9]:
def sharpe_ratio(monthly_returns):
    annualized_rets=((1+monthly_returns).prod())**(12/len(monthly_returns))-1
    annualized_volatility=monthly_returns.std()*np.sqrt(12)
    sharpe=annualized_rets/annualized_volatility
    return pd.Series(sharpe, index=monthly_returns.columns, name="Optimal Weights")
    

#### Sharpe-ratios of the individual factors: 

In [10]:
SR=sharpe_ratio(data)
SR

Market    0.362103
SMB       0.145937
HML       0.229521
RMW       0.387632
CMA       0.449746
UMD       0.442646
ROE       0.656287
IA        0.604648
BAB       0.879851
Name: Optimal Weights, dtype: float64

#### Assuming we keep the optmal theta computed in-sample for the whole period and readjust each month :

In [11]:
returns_param_port=pd.DataFrame(data.dot(theta),columns=['Param_port_rets'])
returns_param_port

,Param_port_rets
Dates,
196702,0.016953
196703,0.007147
196704,0.000776
196705,0.008028
196706,0.003035
...,...
202008,-0.004707
202009,-0.008024
202010,-0.009292


#### Sharpe-Ratio of the parametric portfolio :

In [12]:
SR_parap_port=sharpe_ratio(returns_param_port)
SR_parap_port

Param_port_rets    1.466634
Name: Optimal Weights, dtype: float64

In [13]:
# with long only constraint:

returns_param_port_LO=pd.DataFrame(data.dot(theta_LO),columns=['Param_port_rets'])
SR_parap_port_LO=sharpe_ratio(returns_param_port_LO)
SR_parap_port_LO

Param_port_rets    1.329701
Name: Optimal Weights, dtype: float64

#### --> We observe that the sharpe ratio of the parametric portfolios are much better than the ones of individual factoirs. However these are in-sample results and they are therefore biased. 

<a id='1.4'></a>
### <u> 1.4 Getting the 'real' weights

#### We can derive the weights of the 2000 assets from the factor returns, the computed thetas, the choosen benchmark and the number of assets N=2000, using the following formula:  

$$
w_t(\boldsymbol{\theta}) = w_{b,t} + \frac{(F_{1,t}\theta_1 + F_{2,t}\theta_2 + \cdots + F_{K,t}\theta_K)}{N_t},
$$

where:

- $w_{b,t}$ is determined by the client (could be the market portfolio);
- $F_{k,t}$ is determined by the data on firm characteristics (e.g., size, value, momentum, etc.);
- $N_t$ is determined by how many firms we have data for, so here it is equal 2000;
- $\boldsymbol{\theta} = \{\theta_1, \ldots, \theta_K\}$ is a $K \times 1$ vector, we have found previously


<a id='2'></a>
## <u> 2. Volatility Managed Portfolio

In [14]:
sample=data.loc[197701:]
sample.head()

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
197701,-0.0405,0.0478,0.0426,-0.0053,0.0193,0.0398,-0.021494,0.009953,0.0407
197702,-0.0194,0.0108,0.0050,-0.0013,-0.0019,0.0035,0.012510,0.001210,0.0249
197703,-0.0137,0.0099,0.0102,-0.0032,-0.0009,0.0049,0.002845,-0.001567,0.0123
197704,0.0015,-0.0012,0.0345,-0.0203,0.0116,0.0422,0.003708,0.012277,0.0106
197705,-0.0145,0.0118,0.0084,0.0033,0.0011,0.0201,0.017462,0.002061,0.0104


<a id='2.1'></a>
### <u> 2.1 Volatility timed factors

In [15]:
# We compute the conditional volatilities using the 12m rolling standard deviation of returns. 

conditional_vol=data.loc[197602:].rolling(12).std().dropna()
conditional_var=conditional_vol**2
conditional_vol.head()

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
197701,0.027754,0.027608,0.020366,0.012464,0.015876,0.014985,0.015900,0.012956,0.015225
197702,0.028510,0.020099,0.014606,0.010263,0.011900,0.014986,0.011825,0.011593,0.015462
197703,0.027919,0.019427,0.014356,0.010251,0.011535,0.014955,0.009489,0.010522,0.014321
197704,0.027617,0.019459,0.015712,0.011455,0.011327,0.018371,0.009495,0.010802,0.014074
197705,0.027666,0.018533,0.013663,0.007783,0.010081,0.017575,0.010642,0.010197,0.013604


In [16]:
# We compute our c based on 120 months from Jan 1977 to Jan 1987. This c will be used to compute the out of sample volatility managed factors 
# from Jan 1987 to december 2020

def find_c(conditional_var_sp,sample_f):

    def objective_c(c_vec):
        V_mang = c_vec/conditional_var_sp*sample_f
        volatility_V_mang=V_mang.std()
        abs_diff=abs(volatility_V_mang-sample_f.std())
        return abs_diff.sum()


    initial_weights = np.ones(len(conditional_vol.columns))

    c = minimize(objective_c, initial_weights, method="SLSQP", ).x
    
    return c

c=find_c(conditional_var.loc[:198701],sample.loc[:198701])
c

array([-0.00174016, -0.00031655,  0.00056558, -0.0001913 , -0.0002288 ,
       -0.00076805,  0.00031046, -0.00019168,  0.00036792])

#### We will then use this c for the whole out of sample data, else our managed factors returns will not be consistant between two dates. 

$f_t^\sigma = \frac{c}{\sigma_t(f)} f_t$


In [17]:
volatility_managed_factors=c/conditional_var.loc[:198701]*sample.loc[:198701]
volatility_managed_factors.columns=['mng_MKT','mng_SMB','mng_HML','mng_RMW','mng_CMA','mng_UMD','mng_ROE','mng_IA','mng_BAB']
volatility_managed_factors

,mng_MKT,mng_SMB,mng_HML,mng_RMW,mng_CMA,mng_UMD,mng_ROE,mng_IA,mng_BAB
Dates,,,,,,,,,
197701,0.091497,-0.019852,0.058088,0.006527,-0.017520,-0.136133,-0.026395,-0.011366,0.064602
197702,0.041533,-0.008463,0.013255,0.002361,0.003070,-0.011970,0.027777,-0.001726,0.038320
197703,0.030585,-0.008303,0.027991,0.005826,0.001548,-0.016827,0.009809,0.002713,0.022065
197704,-0.003422,0.001003,0.079043,0.029595,-0.020688,-0.096033,0.012768,-0.020169,0.019688
197705,0.032966,-0.010875,0.025450,-0.010423,-0.002477,-0.049980,0.047870,-0.003799,0.020674
...,...,...,...,...,...,...,...,...,...
198609,0.058171,-0.017210,0.030267,0.000437,-0.021329,0.037639,-0.018958,-0.019840,-0.017094
198610,-0.031175,0.017706,-0.012033,-0.001135,-0.005417,-0.019217,0.004546,-0.000423,-0.008451
198611,-0.008498,0.013537,-0.000646,-0.009673,-0.004553,0.002289,0.005390,-0.002630,-0.005110


<a id='2.2'></a>
### <u> 2.2 MVU for volatility managed portfolio

In [18]:
df_factors=pd.merge(sample,volatility_managed_factors,on='Dates')
df_factors

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB,mng_MKT,mng_SMB,mng_HML,mng_RMW,mng_CMA,mng_UMD,mng_ROE,mng_IA,mng_BAB
Dates,,,,,,,,,,,,,,,,,,
197701,-0.0405,0.0478,0.0426,-0.0053,0.0193,0.0398,-0.021494,0.009953,0.0407,0.091497,-0.019852,0.058088,0.006527,-0.017520,-0.136133,-0.026395,-0.011366,0.064602
197702,-0.0194,0.0108,0.0050,-0.0013,-0.0019,0.0035,0.012510,0.001210,0.0249,0.041533,-0.008463,0.013255,0.002361,0.003070,-0.011970,0.027777,-0.001726,0.038320
197703,-0.0137,0.0099,0.0102,-0.0032,-0.0009,0.0049,0.002845,-0.001567,0.0123,0.030585,-0.008303,0.027991,0.005826,0.001548,-0.016827,0.009809,0.002713,0.022065
197704,0.0015,-0.0012,0.0345,-0.0203,0.0116,0.0422,0.003708,0.012277,0.0106,-0.003422,0.001003,0.079043,0.029595,-0.020688,-0.096033,0.012768,-0.020169,0.019688
197705,-0.0145,0.0118,0.0084,0.0033,0.0011,0.0201,0.017462,0.002061,0.0104,0.032966,-0.010875,0.025450,-0.010423,-0.002477,-0.049980,0.047870,-0.003799,0.020674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198609,-0.0860,0.0228,0.0319,-0.0005,0.0362,-0.0586,-0.038269,0.033936,-0.0244,0.058171,-0.017210,0.030267,0.000437,-0.021329,0.037639,-0.018958,-0.019840,-0.017094
198610,0.0466,-0.0248,-0.0132,0.0013,0.0089,0.0269,0.009132,0.000726,-0.0146,-0.031175,0.017706,-0.012033,-0.001135,-0.005417,-0.019217,0.004546,-0.000423,-0.008451
198611,0.0117,-0.0192,-0.0006,0.0112,0.0061,-0.0032,0.010802,0.003937,-0.0097,-0.008498,0.013537,-0.000646,-0.009673,-0.004553,0.002289,0.005390,-0.002630,-0.005110


In [19]:
round(parametric_MV(df_factors),3)

Market     0.197
SMB       -0.026
HML        0.194
RMW        0.185
CMA       -0.276
UMD       -0.101
ROE       -0.033
IA         0.516
BAB        0.040
mng_MKT    0.159
mng_SMB   -0.122
mng_HML   -0.100
mng_RMW    0.049
mng_CMA   -0.218
mng_UMD   -0.077
mng_ROE    0.257
mng_IA     0.314
mng_BAB    0.041
Name: Optimal Weights, dtype: float64

In [20]:
round(parametric_MV(df_factors,long_only=True),3)

Market     0.000
SMB        0.000
HML        0.000
RMW        0.000
CMA        0.000
UMD        0.008
ROE        0.060
IA         0.000
BAB        0.281
mng_MKT    0.000
mng_SMB    0.000
mng_HML    0.000
mng_RMW    0.000
mng_CMA    0.000
mng_UMD    0.000
mng_ROE    0.200
mng_IA     0.000
mng_BAB    0.452
Name: Optimal Weights, dtype: float64

<a id='3'></a>
## <u> 3. Comparison and Conslusion

In [21]:
def compute_ptf_weights(returns_f, conditional_vol_f):
    conditional_var_f=conditional_vol_f**2
    
    
    volatility_managed_factors=c/conditional_var_f*returns_f
    volatility_managed_factors.columns=['mng_MKT','mng_SMB','mng_HML','mng_RMW','mng_CMA','mng_UMD','mng_ROE','mng_IA','mng_BAB']
    
    df_factors=pd.merge(sample,volatility_managed_factors,on='Dates')
    
    parametric_MV_temp=parametric_MV(df_factors)
    parametric_MV_temp_LO=parametric_MV(df_factors,long_only=True)
    
    return volatility_managed_factors, parametric_MV_temp, parametric_MV_temp_LO

In [22]:
# Using now a rolling window, we compute the weights for the different factors.
#We compute 3 different portfolios:
#    - one standard Long only mean variance for comparison purposes
#    - one parametric portfolio unconstrained
#    - one long only parametric portfolio. 


LO_weights=pd.DataFrame(columns=['Market', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'ROE', 'IA', 'BAB',
       'mng_MKT', 'mng_SMB', 'mng_HML', 'mng_RMW', 'mng_CMA', 'mng_UMD',
       'mng_ROE', 'mng_IA', 'mng_BAB'])
Param_ptf_weights=pd.DataFrame(columns=['Market', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'ROE', 'IA', 'BAB',
       'mng_MKT', 'mng_SMB', 'mng_HML', 'mng_RMW', 'mng_CMA', 'mng_UMD',
       'mng_ROE', 'mng_IA', 'mng_BAB'])

classic_ptf= pd.DataFrame(columns=['Market', 'SMB', 'HML', 'RMW', 'CMA', 'UMD', 'ROE', 'IA', 'BAB'])

for i in range(len(sample)-121):
    returns_rolling=sample.iloc[i:i+120]
    vol_rolling=conditional_vol.iloc[i:i+120]
    weights=compute_ptf_weights(returns_rolling,vol_rolling)
    Param_ptf_weights.loc[sample.index[i+121],:]=weights[1]
    LO_weights.loc[sample.index[i+121],:]=weights[2]
    classic_ptf.loc[sample.index[i+121],:] = parametric_MV(returns_rolling, gamma=1, long_only=True)

In [23]:
Param_ptf_weights

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB,mng_MKT,mng_SMB,mng_HML,mng_RMW,mng_CMA,mng_UMD,mng_ROE,mng_IA,mng_BAB
198702,0.184887,-0.013599,0.197764,0.184758,-0.279901,-0.101439,-0.023491,0.517783,0.030829,0.149448,-0.11785,-0.098849,0.048293,-0.229349,-0.07467,0.254511,0.324912,0.045964
198703,0.196348,-0.027943,0.195104,0.185899,-0.275517,-0.099573,-0.03318,0.516049,0.040463,0.157782,-0.124189,-0.101381,0.049755,-0.215898,-0.074694,0.259136,0.311707,0.040132
198704,0.199776,-0.010587,0.166021,0.149179,-0.271094,-0.113538,-0.003135,0.525666,0.035835,0.166766,-0.109283,-0.094913,0.043222,-0.220567,-0.078074,0.2504,0.3168,0.047528
198705,0.210729,-0.012781,0.188746,0.149421,-0.302388,-0.119271,0.000078,0.557695,0.04108,0.174132,-0.116334,-0.116105,0.037525,-0.247366,-0.082795,0.263213,0.328319,0.046104
198706,0.198188,-0.034577,0.182558,0.156806,-0.241122,-0.098814,-0.007205,0.515363,0.054663,0.16741,-0.12719,-0.145129,0.04991,-0.215077,-0.059298,0.260603,0.291087,0.051825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202008,0.152741,-0.023372,-0.374957,-0.031505,0.119939,-0.332848,-0.02994,0.387301,0.15357,-0.02762,0.052489,0.147277,0.128154,-0.142071,-0.321926,0.34713,0.592815,0.202821
202009,0.151322,-0.023676,-0.4137,-0.067444,0.158192,-0.38402,-0.006667,0.365282,0.20178,-0.032597,0.052632,0.167937,0.128635,-0.102334,-0.356469,0.375318,0.571143,0.214666
202010,0.160864,-0.013729,-0.35363,0.038724,0.245821,-0.295648,-0.011881,0.185058,0.116269,-0.021552,0.045478,0.151488,0.163571,-0.04962,-0.29635,0.315418,0.409331,0.210388
202011,0.179484,-0.014519,-0.444022,-0.00535,0.398898,-0.37038,-0.000461,0.0619,0.158524,-0.030927,0.049071,0.197564,0.169555,0.011293,-0.364826,0.386741,0.360691,0.256764


In [24]:
LO_weights

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB,mng_MKT,mng_SMB,mng_HML,mng_RMW,mng_CMA,mng_UMD,mng_ROE,mng_IA,mng_BAB
198702,0.0,0.0,0.0,0.0,0.0,0.009827,0.047076,0.0,0.259456,0.0,0.0,0.0,0.0,0.0,0.0,0.209627,0.0,0.474013
198703,0.0,0.0,0.0,0.0,0.0,0.0,0.108513,0.0,0.272258,0.0,0.0,0.0,0.0,0.0,0.0,0.221284,0.0,0.397944
198704,0.0,0.0,0.0,0.0,0.0,0.0,0.119194,0.0,0.279748,0.0,0.0,0.0,0.0,0.0,0.0,0.21219,0.0,0.388868
198705,0.0,0.0,0.0,0.0,0.0,0.000589,0.094223,0.0,0.309969,0.0,0.0,0.0,0.0,0.0,0.0,0.197933,0.0,0.397286
198706,0.0,0.0,0.0,0.0,0.0,0.000979,0.084287,0.001887,0.320529,0.0,0.0,0.0,0.0,0.0,0.0,0.196138,0.0,0.396181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202008,0.293438,0.0,0.0,0.0,0.0,0.02267,0.025062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158732,0.0,0.500098
202009,0.275802,0.0,0.0,0.0,0.0,0.021406,0.029923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.155596,0.0,0.517272
202010,0.292464,0.0,0.0,0.0,0.0,0.032821,0.048485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.149082,0.0,0.477148
202011,0.286921,0.0,0.0,0.0,0.0,0.036177,0.000087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.15675,0.0,0.520065


In [25]:
volatility_managed_factors=c/conditional_var*sample
volatility_managed_factors.columns=['mng_MKT', 'mng_SMB', 'mng_HML', 'mng_RMW', 'mng_CMA', 'mng_UMD',
       'mng_ROE', 'mng_IA', 'mng_BAB']

In [26]:
returns_merged=pd.merge(sample,volatility_managed_factors,on='Dates')
returns_merged

,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB,mng_MKT,mng_SMB,mng_HML,mng_RMW,mng_CMA,mng_UMD,mng_ROE,mng_IA,mng_BAB
Dates,,,,,,,,,,,,,,,,,,
197701,-0.0405,0.0478,0.0426,-0.0053,0.0193,0.0398,-0.021494,0.009953,0.0407,0.091497,-0.019852,0.058088,0.006527,-0.017520,-0.136133,-0.026395,-0.011366,0.064602
197702,-0.0194,0.0108,0.0050,-0.0013,-0.0019,0.0035,0.012510,0.001210,0.0249,0.041533,-0.008463,0.013255,0.002361,0.003070,-0.011970,0.027777,-0.001726,0.038320
197703,-0.0137,0.0099,0.0102,-0.0032,-0.0009,0.0049,0.002845,-0.001567,0.0123,0.030585,-0.008303,0.027991,0.005826,0.001548,-0.016827,0.009809,0.002713,0.022065
197704,0.0015,-0.0012,0.0345,-0.0203,0.0116,0.0422,0.003708,0.012277,0.0106,-0.003422,0.001003,0.079043,0.029595,-0.020688,-0.096033,0.012768,-0.020169,0.019688
197705,-0.0145,0.0118,0.0084,0.0033,0.0011,0.0201,0.017462,0.002061,0.0104,0.032966,-0.010875,0.025450,-0.010423,-0.002477,-0.049980,0.047870,-0.003799,0.020674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202008,0.0763,-0.0094,-0.0294,0.0427,-0.0144,0.0051,-0.008682,-0.028534,-0.0399,-0.026914,0.002977,-0.007026,-0.024889,0.008471,-0.001736,-0.005249,0.012385,-0.006700
202009,-0.0363,0.0007,-0.0251,-0.0115,-0.0177,0.0305,0.012255,-0.021992,0.0129,0.012157,-0.000221,-0.009503,0.006866,0.017002,-0.013267,0.007528,0.012839,0.002156
202010,-0.0210,0.0476,0.0403,-0.0060,-0.0053,-0.0303,-0.024671,-0.007387,-0.0201,0.006893,-0.012042,0.011620,0.003544,0.005070,0.012159,-0.013302,0.004329,-0.003312


In [27]:
# we compute the returns for each of our portfolios :
returns_param_ptf= (Param_ptf_weights*returns_merged.loc[198702:,:]).sum(axis=1)
returns_param_ptf_LO= (LO_weights*returns_merged.loc[198702:,:]).sum(axis=1)
returns_classic_ptf= (classic_ptf*sample.loc[198702:,:]).sum(axis=1)

In [28]:
# We now compute the shape ratio for eahc of them
def sharpe_ratio(ret):
    mean_ret=ret.mean()*12
    vol=ret.std()*np.sqrt(12)
    SR= mean_ret/vol
    return SR

SR_LO_MV_port = sharpe_ratio(returns_classic_ptf)
SR_volatility_managed_ptf = sharpe_ratio(returns_param_ptf)
SR_LO_volatility_managed_ptf = sharpe_ratio(returns_param_ptf_LO)

In [29]:
text_summary = f"""
Sharpe Ratios:
--------------
1. Long-Only Mean-Variance Portfolio (SR_LO_MV_port): {SR_LO_MV_port:.4f}
2. Parametric Portfolio (SR_parametric_port): {SR_LO_volatility_managed_ptf:.4f}
3. Long-Only Parametric Portfolio (SR_LO_parametric_port): {SR_volatility_managed_ptf:.4f}
"""

print(text_summary)


Sharpe Ratios:
--------------
1. Long-Only Mean-Variance Portfolio (SR_LO_MV_port): 0.8215
2. Parametric Portfolio (SR_parametric_port): 1.3584
3. Long-Only Parametric Portfolio (SR_LO_parametric_port): -0.0489



The Sharpe ratio of the volatility-managed unconstrained portfolio is negative, indicating that volatility management does not sufficiently compensate for the estimation errors in the expected returns.

However, the constrained volatility-managed portfolio shows a significantly higher Sharpe ratio than the classic constrained mean-variance portfolio. This suggests that volatility timing improves performance. Additionally, these results provide empirical evidence that returns do not always reward risk in the way standard models, such as the CAPM, would predict.

### Warning about limitations

The strategy of timing factors based on their volatilities faces several limitations. First, after accounting for transaction costs, volatility management generally reduces Sharpe ratios for factors other than the market, as the trading costs associated with frequent adjustments can outweigh any potential benefit. In contrast, the volatility-managed market portfolio is more robust to these costs due to its focus on easily arbitraged stocks with low arbitrage risk and few barriers to short-selling.

However, the volatility-timed market strategy performs better mainly during periods of high sentiment, which aligns with the theory that sentiment traders tend to underreact to volatility. Factors other than the market, particularly those that involve small-cap stocks, incur high transaction costs due to large positions in illiquid assets. Moreover, the time-varying leverage of volatility-managed portfolios further exacerbates trading costs, as it forces large trades and significantly increases turnover—up to 15 times higher than for unmanaged portfolios. These issues make implementing a volatility-timing strategy in practice challenging.